In [919]:
# Andrew Shen
# Dr. Yoav Freund
# GOAL: Analysis of easy/hard examples for Dreem dataset of interater agreement using Fleiss Kappa statistic

In [920]:
# Import packages

import pandas as pd

In [921]:
# Current patient to analyze (out of the 80 patients)
file_name = "fc10ee0b-b863-511b-bce8-4dfa7af8ac3a"

In [922]:
# Load in datasets into dataframes

# Read in each file, drop last column (Nan vals) and the last row ("]" value), rename first column as "score"
# dreem1_1 = pd.read_csv("/Users/andrewshen/Desktop/dreem1_1.txt", usecols = [i for i in range(1)])
# dreem1_1.drop(dreem1_1.tail(1).index,inplace=True)
# dreem1_1.columns = ["score"]

# TEST
dreem1_1 = pd.read_json("/Users/andrewshen/Desktop/dreem-learning-evaluation-master/scorers/dodo/scorer_1/" + file_name + ".json")
# dreem1_1.drop(dreem1_1.tail(1).index,inplace=True)
dreem1_1.columns = ["score"]


# dreem1_2 = pd.read_csv("/Users/andrewshen/Desktop/dreem1_2.txt", usecols = [i for i in range(1)])
# dreem1_2.drop(dreem1_2.tail(1).index,inplace=True)
# dreem1_2.columns = ["score"]
dreem1_2 = pd.read_json("/Users/andrewshen/Desktop/dreem-learning-evaluation-master/scorers/dodo/scorer_2/" + file_name + ".json")
# dreem1_2.drop(dreem1_2.tail(1).index,inplace=True)
dreem1_2.columns = ["score"]

# dreem1_3 = pd.read_csv("/Users/andrewshen/Desktop/dreem1_3.txt", usecols = [i for i in range(1)])
# dreem1_3.drop(dreem1_3.tail(1).index,inplace=True)
# dreem1_3.columns = ["score"]
dreem1_3 = pd.read_json("/Users/andrewshen/Desktop/dreem-learning-evaluation-master/scorers/dodo/scorer_3/" + file_name + ".json")
# dreem1_3.drop(dreem1_3.tail(1).index,inplace=True)
dreem1_3.columns = ["score"]

# dreem1_4 = pd.read_csv("/Users/andrewshen/Desktop/dreem1_4.txt", usecols = [i for i in range(1)])
# dreem1_4.drop(dreem1_4.tail(1).index,inplace=True)
# dreem1_4.columns = ["score"]
dreem1_4 = pd.read_json("/Users/andrewshen/Desktop/dreem-learning-evaluation-master/scorers/dodo/scorer_4/" + file_name + ".json")
# dreem1_4.drop(dreem1_4.tail(1).index,inplace=True)
dreem1_4.columns = ["score"]


# dreem1_5 = pd.read_csv("/Users/andrewshen/Desktop/dreem1_5.txt", usecols = [i for i in range(1)])
# dreem1_5.drop(dreem1_5.tail(1).index,inplace=True)
# dreem1_5.columns = ["score"]
dreem1_5 = pd.read_json("/Users/andrewshen/Desktop/dreem-learning-evaluation-master/scorers/dodo/scorer_5/" + file_name + ".json")
# dreem1_5.drop(dreem1_5.tail(1).index,inplace=True)
dreem1_5.columns = ["score"]

num_vals = dreem1_1.size

In [923]:
# dreem1_1
num_vals

951

In [924]:
# Create master dataframe
dreem1_master = dreem1_1.copy()
dreem1_master["score2"] = dreem1_2["score"]
dreem1_master["score3"] = dreem1_3["score"]
dreem1_master["score4"] = dreem1_4["score"]
dreem1_master["score5"] = dreem1_5["score"]

In [925]:
# dreem1_master

In [926]:
# Background variables to calculate

# i: rater id, from 1-5
# j: rater id, from 1-5
# k: example/sample id, from 0-1191
# X(i,j,k): random variable equal to 1 if labeler i and j agree on example k, otherwise 0
# AG: average agreement rate, average of X(i,j,k) for all i,j,k
# Y(i,j,k1,k2): random variable equal to 1 if two labelers i and j agree on random exammples k1 and k2 (with k1 from i and k2 from j), otherwise 0
# BACK: 

# Use statistic AG-BACK to determine easy vs hard examples

In [927]:
# Calculate X

# Function to facilitate calculation of X for all possible combinations of i,j,k
# Dictionary to easily access dreem datasets based on an id number
raterID_dict = {1:dreem1_1, 2:dreem1_2, 3:dreem1_3, 4:dreem1_4, 5:dreem1_5}
def get_x(i,j,k):
    """
    GOAL: Return whether two labelers agree on an example k
    Input: i (rater id), j (rater id), k (sample number)
    Output: 1 (if agree), 0 (if don't agree)
    """
    i_df = raterID_dict[i]
    j_df = raterID_dict[j]
    i_val = i_df.loc[k]["score"]
    j_val = j_df.loc[k]["score"]
    #print(i_val)
    #print(j_val)
    if(i_val == j_val):
        return 1
    return 0

In [928]:
# Calculate AG

# Iterate over all combinations of i and j (not including when i equals j), for each get values of k
total_agreement = 0
total_combos = 0
# Dictionary with key as rater pairings and value as AG between the two raters
# Number of pairings is C(5,2)
pairings = [[1,2], [1,3], [1,4], [1,5], [2,3], [2,4], [2,5], [3,4], [3,5], [4,5]]
num_agreement_dict = {}
# AG_dict = {}

# Populate "num_agreement_dict" to find agreement value
for k in range(0,num_vals):
    curr_agreement_count = 0 
    for pair in pairings:
        i = pair[0]
        j = pair[1]
        curr_agreement_count += get_x(i,j,k)
    num_agreement_dict[k] = curr_agreement_count

# Old code that finds "total_agreement" and "curr_agreement"
# for pair in pairings:
#     i = pair[0]
#     j = pair[1]
#     if(i!=j):
#         curr_id = str(i) + "," + str(j)
#         curr_agreement = 0
#         curr_combos = 0
#         for k in range(0,1192):
#             total_agreement += get_x(i,j,k)
#             total_combos += 1
#             curr_agreement += get_x(i,j,k)
#             curr_combos += 1
#             #print("K iteration for: " + str(i) + " " + str(j) + " " + str(k))
#         AG_dict[curr_id] = curr_agreement/curr_combos

In [929]:
# num_agreement_dict

In [930]:
# Add values of "num_agreement_dict" to "dreem1_master"
dreem1_master["num_agreement"] = num_agreement_dict.values()

In [931]:
# dreem1_master

In [932]:
# Export final "dreem1_master" dataframe
outfile_name = "dreem_analysis_out/" + file_name + "_out.csv"
dreem1_master.to_csv(outfile_name, mode='a')

In [933]:
# AG_total = total_agreement/total_combos # 0.5445469

In [934]:
# AG_dict

In [935]:
# Calculate Y

# Function to facilitate calculation of Y for all possible combinations of i,j,k1,k2
#def get_y(